**Содержание**

Этот набор данных содержит изображения, относящиеся к набору данных EuroSat. В нём есть 2 папки, а именно:

EuroSAT → Содержит изображения RGB, собранные из набора данных Sentinel.

EuroSATallBands → Содержит файлы .tif, в которых представлены все диапазоны спектра, собранные со спутника Sentinel-2.

Каждое изображение имеет размер 64x64 пикселя с шагом 10 м. Все они были получены со спутника Sentinel-2

In [ ]:
!pip install kagglehub

import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets, models
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import kagglehub
from PIL import ImageFile, Image

Мне необходима именно папка Forest, потому что до этого я пыталась обучить на всех данных и у меня ушло 12 часов только на 3 эпохи... Решила сократить.

In [ ]:
import os
import shutil

path = kagglehub.dataset_download("apollo2506/eurosat-dataset")
forest_path = "/root/.cache/kagglehub/datasets/apollo2506/eurosat-dataset/versions/6/EuroSAT/Forest"
new_forest_path = os.path.join(forest_path, "forest")  # Создаем подпапку "forest"

os.makedirs(new_forest_path, exist_ok=True)

for filename in os.listdir(forest_path):
        try:
            shutil.move(os.path.join(forest_path, filename), new_forest_path)  # Перемещаем файлы
        except Exception as e:
            print(f"Ошибка перемещения файла {filename}: {e}")

            forest_path = os.path.join(dataset_path, "EuroSAT", "Forest", "forest") # Путь к подпапке "forest"

100%|██████████| 2.04G/2.04G [00:24<00:00, 89.2MB/s]

Extracting files...


In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_data = datasets.ImageFolder(root=forest_path, transform=transform)
val_data = datasets.ImageFolder(root=forest_path, transform=transform)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)

print(f"Классы: {train_data.classes}") # Forest
num_classes = len(train_data.classes)

Классы: ['forest']


In [ ]:
# Определение устройства (GPU или CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Загрузка предобученной модели ResNet50
model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)

# Заморозка весов предобученной части модели
for param in model.parameters():
    param.requires_grad = False

# Замена последнего слоя на новый с одним выходом (для бинарной классификации)
model.fc = nn.Linear(model.fc.in_features, 1)

# Перемещение модели на устройство
model = model.to(device)

# Определение функции потерь и оптимизатора
criterion = nn.BCEWithLogitsLoss()  # Бинарная кросс-энтропия
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

Using device: cpu


Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 98.8MB/s]


In [ ]:
# Количество эпох
num_epochs = 10

# Обучение модели
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs.squeeze(), labels.float())
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

print("Training Finished!")

Epoch 1, Loss: 0.05800846001093692
Epoch 2, Loss: 0.006273643004688177
Epoch 3, Loss: 0.003528203444921986
Epoch 4, Loss: 0.002352270727452049
Epoch 5, Loss: 0.0018778150198991074
Epoch 6, Loss: 0.0013437407447936687
Epoch 7, Loss: 0.0011011943325742169
Epoch 8, Loss: 0.0009718899555662845
Epoch 9, Loss: 0.0007596531922512866


In [ ]:
# Сохранение весов модели
torch.save(model.state_dict(), "forest_resnet50.pth")
print("The model is saved!")

The model is saved!


In [ ]:
# Список возможных состояний леса
forest_states = [
    "Полностью здоровый лес",
    "Лес с небольшими проблемами (небольшие участки вырубки, признаки болезней)",
    "Лес с умеренными проблемами (заметные участки вырубки, распространение болезней)",
    "Лес с серьезными проблемами (значительные вырубки, массовые заболевания)",
    "Лес, требующий немедленного восстановления (практически полное отсутствие растительности)"
]

def predict_forest_state(image_path): # Предсказывает состояние леса на основе изображения.

    # Загрузка и преобразование изображения
    image = Image.open(image_path).convert("RGB") # Важно преобразовать в RGB
    image = transform(image).unsqueeze(0).to(device)

    # Предсказание
    with torch.no_grad():
        output = model(image)

        # Предполагаем, что модель возвращает вероятности для каждого класса forest_states
        probabilities = torch.softmax(output, dim=1) # Применяем softmax, чтобы получить вероятности
        predicted_class = torch.argmax(probabilities).item() # Получаем индекс класса с наибольшей вероятностью
        confidence = probabilities[0][predicted_class].item() # Получаем уверенность в предсказании

    # Формирование рекомендаций
    state = forest_states[predicted_class]
    confidence_percentage = round(confidence * 100, 2)
    recommendation = f"Рекомендация: {state} (уверенность: {confidence_percentage}%)."

    # Дополнительные рекомендациит:
    if "вырубки" in state.lower():
        recommendation += " Рекомендуется провести мониторинг участков вырубки и принять меры по лесовосстановлению."
    elif "болезни" in state.lower():
        recommendation += " Необходима проверка на наличие заболеваний и принятие мер по их лечению."
    elif confidence < 0.7:
        recommendation = f"Недостаточная уверенность в оценке. Рекомендуется дополнительный анализ."
    return recommendation

# Пример использования функции
image_path = "felled_forest.jpg"
print(predict_forest_state(image_path))

Рекомендация: Полностью здоровый лес (уверенность: 100.0%).


Это лишь набросок рекомендаций. Но чтобы рекомендации были более точными, можно добавить анализ дополнительных параметров:

Площадь поражения;
площадь вырубки или деградации;
Интенсивность изменений. Например, если модель может определить степень деградации (слабая, средняя, сильная), рекомендации можно адаптировать под это;
Временные изменения. Отследить динамику изменений (например, скорость вырубки или восстановления).
Но отсюда и проблема... Сложно получить спутниковые снимки с временными рядами и дообучить модель. Было бы здорово дообучить её на данных, которые включают:

Метки классов с дополнительной информацией. Например, не просто "Лес", а "Лес с хорошим покровом", "Лес деградированный", "Лес после пожара" (как раз с этого и начали).

И уже для каждого класса можно добавить текстовое описание рекомендаций, и модель сможет учиться предсказывать их напрямую. Например, "Лес в хорошем состоянии. Рекомендуется продолжить мониторинг.", "Лес деградирован. Рекомендуется восстановить растительный покров.", "Обнаружены следы пожаров. Необходимо восстановить экосистему.".

Но нужны хорошие размеченные данные...

Если совсем далеко заглядывать, то в дальнейшем наладить механизм обратной связи: Пользователь (например, эколог или лесник) может оценить, насколько рекомендации были полезны. Эти данные можно использовать для дальнейшего дообучения модели.